In [ ]:
import os
import requests
from dotenv import load_dotenv
from pathlib import Path
import time
import json

# Load environment variables
load_dotenv()

# Get API key from environment
api_key = os.getenv('TRIPO3D_API_KEY')
if not api_key:
    raise ValueError("Please set TRIPO3D_API_KEY in your .env file")


In [ ]:
# Function to upload image and start processing
def process_image(image_path):
    # API endpoints
    upload_url = 'https://platform.tripo3d.ai/api/v1/reconstruction/upload'
    process_url = 'https://platform.tripo3d.ai/api/v1/reconstruction/process'
    
    # Headers for authentication
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    
    # Upload image
    with open(image_path, 'rb') as f:
        files = {'file': f}
        response = requests.post(upload_url, headers=headers, files=files)
        response.raise_for_status()
        upload_data = response.json()
        
    # Start processing
    process_data = {
        'imageUrls': [upload_data['url']],
        'outputFormat': 'glb',  # You can change this to 'obj' if needed
        'textured': True
    }
    
    response = requests.post(process_url, headers=headers, json=process_data)
    response.raise_for_status()
    return response.json()


In [ ]:
# Function to check processing status and download result
def check_status_and_download(task_id, output_dir):
    status_url = f'https://platform.tripo3d.ai/api/v1/reconstruction/{task_id}'
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    
    while True:
        response = requests.get(status_url, headers=headers)
        response.raise_for_status()
        status_data = response.json()
        
        if status_data['status'] == 'completed':
            # Download the result
            download_url = status_data['outputUrl']
            model_response = requests.get(download_url)
            output_path = Path(output_dir) / f'model_{task_id}.glb'
            with open(output_path, 'wb') as f:
                f.write(model_response.content)
            print(f'Model downloaded to {output_path}')
            break
        elif status_data['status'] == 'failed':
            print('Processing failed:', status_data.get('error', 'Unknown error'))
            break
        
        print('Processing... Current status:', status_data['status'])
        time.sleep(10)  # Wait 10 seconds before checking again


In [ ]:
# Process the test image
image_path = 'test.png'
output_dir = '.'

try:
    # Start processing
    result = process_image(image_path)
    task_id = result['taskId']
    print(f'Processing started with task ID: {task_id}')
    
    # Check status and download result
    check_status_and_download(task_id, output_dir)
    
except requests.exceptions.RequestException as e:
    print('Error occurred:', str(e))
